In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [3]:
first, last = section_range_lookup['MD589']
dms = dict([(sc, DataManager(stack='MD589', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(first, last)])

In [6]:
sys.path.append('/oasis/projects/nsf/csd395/yuncong/mxnet/python')

In [7]:
from FeatureExtractor import *

In [9]:
dm = dms[120]

In [11]:
from enum import Enum

class ProposalType(Enum):
    GLOBAL = 'global'
    LOCAL = 'local'
    FREEFORM = 'freeform'
    ALGORITHM = 'algorithm'
    
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [54]:
landmark_images = {}
adjusted_vertices = {}

sec = 120

print sec

usr, ts, suffix, props = dm.load_proposal_review_result('yuncong', 'latest', 'consolidated')

vertices = np.array(props[1]['vertices'])

xmin, ymin  = np.min(vertices, axis=0).astype(np.int) - 300
xmax, ymax  = np.max(vertices, axis=0).astype(np.int) + 300

dms[sec]._load_image(['rgb'])
viz = dms[sec].image_rgb[ymin:ymax+1, xmin:xmax+1].copy()
landmark_images[sec] = viz
del dms[sec].image_rgb

adjusted_vertices[sec] = vertices - (xmin, ymin)

patch_size = 224
half_size = patch_size/2
stride = 112

patches_allSections = {}
patches = []

h, w = landmark_images[sec].shape[:2]
ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride))
polygon_path = Path(adjusted_vertices[sec])
for x, y in np.c_[xs.flat, ys.flat]:
    if np.all(polygon_path.contains_points([(x-half_size, y-half_size),(x+half_size, y-half_size),
                                            (x+half_size, y+half_size),(x-half_size, y+half_size)])):
        patch = landmark_images[sec][y-half_size:y+half_size, x-half_size:x+half_size].copy()
        patches.append(patch)
patches_allSections[sec] = patches


all_patches = [p for sec, patches in patches_allSections.iteritems() for p in patches]
print len(all_patches), 'patches'

120
1 patches


In [56]:
all_patches_arr = np.array([img_as_ubyte(p) for p in all_patches], np.uint8)

In [57]:
model_dir = '/oasis/projects/nsf/csd395/jiaxuzhu/model/vgg16'

fe = FeatureExtractor(model_dir, batch_size=16, ctx='cpu')

images = all_patches_arr.transpose(0, 3, 1, 2)
print images.shape

features = fe.extract(images)
print features.shape
# np.save(os.path.join(data_dir, '%s_features.npy' % name), features)

(37, 3, 224, 224)
(37, 4096)


In [58]:
label1 = '12N'
label2 = 'Gr'

In [59]:
coef = np.load('%s_%s_linear_svm_coef.npy' % (label1, label2))
intercept = np.load('%s_%s_linear_svm_intercept.npy' % (label1, label2))
selected_feature_indices = np.load('%s_%s_linear_svm_selectedFeatureIndices.npy' % (label1, label2))

In [60]:
y = np.dot(features[:, selected_feature_indices], np.squeeze(coef)) + intercept
Y = np.sign(y)

In [61]:
Y

array([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.])